In [1]:
import cv2
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import Model, load_model
from project_utils import prediction_generator

Using TensorFlow backend.


In [2]:
# image size
IMG_W = 320
IMG_H = 320
IMG_C = 3

# train image paths and their int_class
train_img_paths = np.array(open('MURA-v1.1/train_image_paths.csv', 'r').readlines())

# validation image paths and their int_class
valid_img_paths = np.array(open('MURA-v1.1/valid_image_paths.csv', 'r').readlines())

In [4]:
BATCH_SIZE = 32

In [3]:
base_model = load_model('densenet2_066.hdf5')
model = Model(base_model.input, base_model.layers[-2].output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 326, 326, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 160, 160, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 160, 160, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [4]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.save('embed_model64.hdf5')

In [3]:
model = load_model('embed_model64.hdf5')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 326, 326, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 160, 160, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 160, 160, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [5]:
train_embeds = model.predict_generator(prediction_generator(train_img_paths, BATCH_SIZE), steps = len(train_img_paths)/BATCH_SIZE)

In [6]:
np.save('train_img_embeds64', train_embeds)

In [7]:
valid_embeds = model.predict_generator(prediction_generator(valid_img_paths, BATCH_SIZE), steps = len(valid_img_paths)/BATCH_SIZE)

In [8]:
np.save('valid_img_embeds64', valid_embeds)